In [6]:
def grep(pattern):
    print("start grep for", pattern)

    while True:
        s = yield
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))
            

grep_gen = grep("python")

In [7]:
print(next(grep_gen))

start grep for python
None


In [8]:
grep_gen.send("python")
grep_gen.send("py")

found! python
no python in py


In [9]:
grep_gen.throw(ValueError("WRONG"))

ValueError: WRONG

In [5]:
next(grep_gen)

TypeError: argument of type 'NoneType' is not iterable

In [10]:
import asyncio

In [11]:
@asyncio.coroutine
def grep_coro(pattern):
    print("start grep for", pattern)

    while True:
        s = yield
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))
            

grep_gen = grep_coro("python")

/var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_61041/1487714701.py:2: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def grep_coro(pattern):


In [28]:
async def count(n):
    k = n
    print("START count", n)
    while n > 0:
        n -= 1
        await asyncio.sleep(1)  # 10 sec
    print("END count", k)


async def block(n):
    n = 1
    while n > 100:
        n -= 1
        n *= n
        #await asyncio.sleep(0)

In [14]:
await count(10)

<coroutine object count at 0x1140a8820>

In [15]:
asyncio.run(count(10))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [19]:
import time

t1 = time.time()
print("begin")

await count(10)
await count(6)

print("end", time.time() - t1)

begin
end 16.041203260421753


In [23]:
await count(10)

START count 10


In [29]:
import time

t1 = time.time()
print("begin")

task1 = asyncio.create_task(count(10))
task2 = asyncio.create_task(count(6))

print("end", time.time() - t1)

begin
end 0.0006361007690429688
START count 10
START count 6
END count 6
END count 10


In [32]:
import time

t1 = time.time()
print("begin")

task1 = asyncio.create_task(count(10))
task2 = asyncio.create_task(count(6))

await task1
await task2

print("end", time.time() - t1)

begin
START count 10
START count 6
END count 6
END count 10
end 10.008776903152466


In [36]:
import time

t1 = time.time()
print("begin")

task1 = asyncio.create_task(count(10))
task2 = asyncio.create_task(count(6))

await task2
print("task2 ready")

task1.cancel()
await task1

print("end", time.time() - t1)

begin
START count 10
START count 6
END count 6
task2 ready


CancelledError: 

In [35]:
await c

RuntimeError: cannot reuse already awaited coroutine

In [37]:
async def count(n):
    k = n
    print("START count", n)
    while n > 0:
        print(f"count_{k}: {n}")
        n -= 1
        await asyncio.sleep(1)  # 10 sec
    print("END count", k)


t1 = time.time()

tasks = [
    asyncio.create_task(count(i))
    for i in range(2, 10, 2)
]
gath = asyncio.gather(*tasks)
print("create_gath")
await gath

print("time", time.time() - t1)

create_gath
START count 2
count_2: 2
START count 4
count_4: 4
START count 6
count_6: 6
START count 8
count_8: 8
count_2: 1
count_4: 3
count_6: 5
count_8: 7
END count 2
count_4: 2
count_6: 4
count_8: 6
count_4: 1
count_6: 3
count_8: 5
END count 4
count_6: 2
count_8: 4
count_6: 1
count_8: 3
END count 6
count_8: 2
count_8: 1
END count 8
time 8.007362842559814


In [38]:
async def count(n):
    k = n
    print("START count", n)
    while n > 0:
        print(f"count_{k}: {n}")
        n -= 1
        await asyncio.sleep(1)  # 10 sec
    print("END count", k)


t1 = time.time()

tasks = [
    asyncio.create_task(count(7)),
    asyncio.create_task(count(3)),
    asyncio.create_task(count(9)),
    asyncio.create_task(count(6)),
]

for task in asyncio.as_completed(tasks):
    print("time task", time.time() - t1)
    res = await task
    print("res", res, task)


print("time", time.time() - t1)

time task 0.00019478797912597656
START count 7
count_7: 7
START count 3
count_3: 3
START count 9
count_9: 9
START count 6
count_6: 6
count_7: 6
count_3: 2
count_9: 8
count_6: 5
count_7: 5
count_3: 1
count_9: 7
count_6: 4
count_7: 4
END count 3
count_9: 6
count_6: 3
res None <coroutine object as_completed.<locals>._wait_for_one at 0x113f3af80>
time task 3.003438711166382
count_7: 3
count_9: 5
count_6: 2
count_7: 2
count_9: 4
count_6: 1
count_7: 1
count_9: 3
END count 6
res None <coroutine object as_completed.<locals>._wait_for_one at 0x1143f5ee0>
time task 6.007556676864624
END count 7
count_9: 2
res None <coroutine object as_completed.<locals>._wait_for_one at 0x113f3af80>
time task 7.008103847503662
count_9: 1
END count 9
res None <coroutine object as_completed.<locals>._wait_for_one at 0x1143f5ee0>
time 9.053854942321777


In [39]:
asyncio.get_event_loop()

<_UnixSelectorEventLoop running=True closed=False debug=False>

In [49]:
import aiohttp


URL = "https://docs.python.org/3/library/asyncio.html"


async def fetch_url(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            data = await resp.read()
            print(resp.status, len(data))

            return len(data)
        

t1 = time.time()

await fetch_url(URL)

print("time", time.time() - t1)

200 17556
time 0.17046880722045898


In [69]:
import aiohttp


URL = "https://docs.python.org/3/library/asyncio.html"
N = 100


async def fetch_url(url, session, lock):
    async with lock:
        async with session.get(url) as resp:
            data = await resp.read()
            #print(resp.status, len(data))

            return len(data)

    
async def fetch_batch_urls(urls):
    lock = asyncio.Lock()
    
    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch_url(url, session, lock))
            for url in urls
        ]
        await asyncio.gather(*tasks)


t1 = time.time()

urls = [URL] * N
await fetch_batch_urls(urls)

print("time", time.time() - t1)

time 4.339529037475586


In [74]:
import aiohttp


URL = "https://docs.python.org/3/library/asyncio.html"
N = 100


async def fetch_url(url, session, lock):
    async with lock:
        async with session.get(url) as resp:
            data = await resp.read()
            #print(resp.status, len(data))

            return len(data)

    
async def fetch_batch_urls(urls):
    lock = asyncio.Semaphore(20)
    
    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(fetch_url(url, session, lock))
            for url in urls
        ]
        await asyncio.gather(*tasks)


t1 = time.time()

urls = [URL] * N
await fetch_batch_urls(urls)

print("time", time.time() - t1)

time 0.3565223217010498


In [86]:
import aiohttp


URL = "https://docs.python.org/3/library/asyncio.html"
N = 100
WORKERS = 20


async def fetch_url(url, session):
    async with session.get(url) as resp:
        data = await resp.read()
        #print(resp.status, len(data))

        return len(data)


async def worker(queue, session, num):
    while True:
        url = await queue.get()

        try:
            res = await fetch_url(url, session)
            #print(f"worker_{num}", res)
        finally:
            queue.task_done()

    
async def fetch_batch_urls(queue, workers):
    async with aiohttp.ClientSession() as session:
        tasks = [
            asyncio.create_task(worker(queue, session, i))
            for i in range(workers)
        ]
        await queue.join()
        
        for task in tasks:
            task.cancel()


t1 = time.time()

urls_queue = asyncio.Queue()
for _ in range(N):
    await urls_queue.put(URL)


await fetch_batch_urls(urls_queue, WORKERS)

print("time", time.time() - t1)

time 0.40215301513671875
